In [2]:
import numpy as np
import pandas as pd

In [3]:
data = np.loadtxt("/content/M_2.txt")
df = pd.DataFrame(data)

In [4]:
df = df.drop(columns=1)
df.iloc[:, 0].unique()

array([1., 2., 3., 4., 5., 6., 7.])

## 1. Evalúa un modelo perceptrón multicapa con validación cruzada utilizando al menos 5 capas de 20 neuronas.



In [5]:
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

In [6]:
y = df.iloc[:, 0]
x = df.iloc[:, 1:]
n_features = x.shape[1]
print(n_features)

targets = y.unique()
n_clases = len(targets)
n_clases

630


7

In [7]:
# Train classifier with all the available observations
clf = MLPClassifier(hidden_layer_sizes=(20, 20, 20, 20, 20), max_iter=10000)
# hidden_layer_sizes controls the number of neurons of each hidden layer.
clf.fit(x, y)


# 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True)
cv_y_test = []
cv_y_pred = []

for train_index, test_index in kf.split(x, y):
    # Training phase
    x_train = x.iloc[train_index]
    y_train = y.iloc[train_index]
    clf_cv = MLPClassifier(hidden_layer_sizes=(20, 20, 20, 20, 20), max_iter=10000)
    clf_cv.fit(x_train, y_train)

    # Test phase
    x_test = x.iloc[test_index]
    y_test = y.iloc[test_index]
    y_pred = clf_cv.predict(x_test)
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)

print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))

# End of file


              precision    recall  f1-score   support

         1.0       0.92      0.89      0.90        90
         2.0       0.75      0.74      0.75        90
         3.0       0.96      0.96      0.96        90
         4.0       0.92      0.88      0.90        90
         5.0       0.74      0.71      0.72        90
         6.0       0.73      0.73      0.73        90
         7.0       0.88      0.99      0.93        90

    accuracy                           0.84       630
   macro avg       0.84      0.84      0.84       630
weighted avg       0.84      0.84      0.84       630



Los resultados de este modelo (Red neuronal perceptrón multicapa) muestran un rendimiento generalmente sólido en la tarea de clasificación. La métrica de precisión varía para cada clase, lo que indica la capacidad del modelo para identificar correctamente cada categoría. En promedio, el modelo alcanza una precisión del 85%, lo que significa que, en promedio, el 85% de las predicciones son correctas. Además, se observa que el modelo logra un equilibrio razonable entre precisión y exhaustividad (recall) en la mayoría de las categorías, con valores de F1-score que indican un buen equilibrio entre precisión y recall.

## 2. Evalúa un modelo perceptrón multicapa con validación cruzada, pero encontrando el número óptimo de capas y neuronas de la red


In [8]:
df.head()

,0,2,3,4,5,6,7,8,9,10,...,622,623,624,625,626,627,628,629,630,631
0,1.0,0.179021,0.690598,0.106307,-0.874411,0.534781,0.944589,1.042887,-0.447626,0.670869,...,0.707322,-0.413054,1.579281,0.577339,-0.210420,-0.096947,1.914895,0.164310,2.423030,0.940309
1,1.0,-0.392490,1.380636,-0.506769,-1.342105,-1.024075,-1.476238,1.439298,-0.912754,-1.030290,...,-0.183109,-0.837541,1.669477,-1.687489,-1.435424,-2.126351,0.812081,0.534858,2.358621,-1.840876
2,1.0,-0.264944,0.291899,-1.174263,-1.706030,-0.831943,-0.919606,0.322527,-1.214675,-0.565001,...,-0.552510,-1.277416,0.588301,-1.513276,-1.719922,-1.631825,0.286730,-0.286559,1.273157,-1.124130
3,1.0,-0.002269,0.714463,-0.928275,-0.546158,-0.803237,0.024654,0.596932,-0.345179,-0.615986,...,0.633401,-1.269364,0.296169,-0.454143,-0.024468,-0.546091,1.360139,-0.979954,1.461883,-0.152339
4,1.0,-0.138267,0.311905,0.501269,-1.545965,-0.797499,-0.582358,0.025408,0.376933,-0.038809,...,1.184483,-0.732250,2.106686,0.232933,-0.312752,-0.625347,1.790285,0.618412,3.044807,-0.063636


In [9]:
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import classification_report


In [10]:
################################################################################
#se crea un objeto MLPClassifier que representa un modelo MLP con dos capas ocultas,
#cada una con 100 neuronas. Se establece max_iter en 10000,
#lo que significa que el modelo puede entrenar durante un máximo de 10,000 épocas.
################################################################################

y = df.iloc[:, 0]
x = df.iloc[:, 1:]
#se entrena el modelo con todos los datos
clf = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=10000)
clf.fit(x, y)

################################################################################
# Evaluamos utilizando las mismas arquitecturas(2 capas de 100 neuronas) y usando validación cruzada para evaluar
################################################################################
y_pred = cross_val_predict(MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=10000), x, y)
print(classification_report(y, y_pred))

################################################################################
# Realizamos una búsqueda de hiperparámetros para encontrar la mejor combinación de capas y neuronas en la red MLP.
################################################################################

num_layers = np.arange(1, 5, 1) #número de capas (de 1 a 4)
num_neurons = np.arange(10, 60, 10) #número de neuronas por capa (de 10 a 50 en incrementos de 10)
layers = [] # contiene todas las combinaciones posibles de capas y neuronas
for l in num_layers:
    for n in num_neurons:
        layers.append(l*[n])

#Se utiliza GridSearchCV para encontrar la mejor arquitectura de red utilizando la validación cruzada con 5 folds.
clf = GridSearchCV(MLPClassifier(max_iter=10000), {'hidden_layer_sizes': layers}, cv=5)
clf.fit(x, y)
print(clf.best_estimator_)

################################################################################
# Evaluate model finding the optimal number of layers and neurons
################################################################################

#se evalúa el modelo MLP con la arquitectura óptima encontrada en la etapa anterior.
clf = GridSearchCV(MLPClassifier(max_iter=10000), {'hidden_layer_sizes': layers}, cv=5)
y_pred = cross_val_predict(clf, x, y, cv=5)
print(classification_report(y, y_pred))

#----------------------------------------------------------------------------------
# End of file
#----------------------------------------------------------------------------------


              precision    recall  f1-score   support

         1.0       0.90      0.90      0.90        90
         2.0       0.70      0.77      0.73        90
         3.0       0.94      0.92      0.93        90
         4.0       0.93      0.87      0.90        90
         5.0       0.81      0.74      0.77        90
         6.0       0.78      0.77      0.77        90
         7.0       0.88      0.96      0.91        90

    accuracy                           0.85       630
   macro avg       0.85      0.85      0.85       630
weighted avg       0.85      0.85      0.85       630

MLPClassifier(hidden_layer_sizes=[50, 50, 50], max_iter=10000)
              precision    recall  f1-score   support

         1.0       0.87      0.92      0.90        90
         2.0       0.70      0.78      0.74        90
         3.0       0.94      0.93      0.94        90
         4.0       0.92      0.87      0.89        90
         5.0       0.80      0.77      0.78        90
         6.0   


Informe del Modelo Inicial (Arquitectura Inicial):

Bueno, aquí tenemos los resultados del primer modelo MLP que entrené. Este modelo tiene dos capas ocultas, cada una con 100 neuronas. El informe muestra cómo se desempeñó en mi conjunto de datos.
Mi conjunto de datos tiene múltiples clases numeradas del 1 al 7, por lo que el informe muestra métricas como precisión, recall y puntaje F1 para cada clase individualmente.
Los números en el informe representan cómo el modelo predijo las etiquetas en comparación con las etiquetas reales. Por ejemplo, la precisión promedio ponderada del 85% significa que, en promedio, el modelo acertó el 85% de las predicciones en mi conjunto de datos.


Informe del Modelo Óptimo (Arquitectura Óptima):

Aquí están los resultados de mi segundo modelo MLP, que tiene una arquitectura diferente. Encontré esta arquitectura durante la búsqueda de hiperparámetros y resultó en tres capas ocultas, cada una con 50 neuronas.
Al igual que en el primer informe, ves métricas como precisión, recall y puntaje F1 para cada clase. Esto me ayuda a entender cómo el modelo se desempeña en cada categoría de mi conjunto de datos.
La precisión promedio ponderada en este caso es del 84%, lo que significa que este modelo clasifica correctamente el 84% de las muestras en mi conjunto de datos.



Ambos modelos tienen resultados bastante similares, pero el modelo óptimo utiliza menos neuronas y capas para lograr un rendimiento comparable, lo que podría ser una ventaja debido a su simplicidad. En última instancia, la elección entre la arquitectura inicial y la óptima depende de mis necesidades y preferencias personales, y si valoro la simplicidad o la complejidad en mi modelo.

## 4. Prepara el modelo perceptrón multicapa:
Opten los hiperparámetros óptimos de capas y neuronas de la red.
Con los hiperparámetros óptimos, ajusta el modelo con todos los datos.

In [15]:
y = df.iloc[:, 0]
x = df.iloc[:, 1:]

best_mlp = MLPClassifier(hidden_layer_sizes=[50, 50, 50], max_iter=10000)  # Utiliza los valores óptimos
best_mlp.fit(x, y)

# 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True)
cv_y_test = []
cv_y_pred = []

for train_index, test_index in kf.split(x, y):
    # Training phase
    x_train = x.iloc[train_index]
    y_train = y.iloc[train_index]
    clf_cv = MLPClassifier(hidden_layer_sizes=[50, 50, 50], max_iter=10000)  # Utiliza los valores óptimos
    clf_cv.fit(x_train, y_train)

    # Test phase
    x_test = x.iloc[test_index]
    y_test = y.iloc[test_index]
    y_pred = clf_cv.predict(x_test)
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)

# Genera un informe de clasificación basado en los resultados de la validación cruzada
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))


              precision    recall  f1-score   support

         1.0       0.90      0.89      0.89        90
         2.0       0.72      0.79      0.76        90
         3.0       0.97      0.93      0.95        90
         4.0       0.86      0.92      0.89        90
         5.0       0.81      0.82      0.82        90
         6.0       0.82      0.72      0.77        90
         7.0       0.93      0.92      0.93        90

    accuracy                           0.86       630
   macro avg       0.86      0.86      0.86       630
weighted avg       0.86      0.86      0.86       630



## 4. Contesta lo siguiente:

**¿Observas alguna mejora importante al optimizar el tamaño de la red? ¿Es el resultado que esperabas? Argumenta tu respuesta.**

Observando los resultados, si hubo una mejora mínima al optimizar el tamaño de la red. Aunque ajustamos la arquitectura para tener tres capas ocultas en lugar de dos, y redujimos el número de neuronas a 50 en cada capa en lugar de 100, las métricas de evaluación no mostraron una diferencia significativa en el rendimiento del modelo. La precisión promedio ponderada en ambos casos era muy similar, alrededor del 84-86%. Esto fue un poco decepcionante, ya que esperaba una mejora más notable en el rendimiento al ajustar la arquitectura de la red.

Por otro lado, también es importante notar que el número de neuronas por capa en la optimizacion de los hiperparametros llegaba hasta 50, y no a 100 como lo hicimos con el modelo incial. Esto es porque buscar el número óptimo de capas y neuronas de la red es muy costoso y tardado y fue necesario reducir el numero de neuronas a la mitad. De igual forma, corriendo un modelo de dos capas con 50 neuronas nos dieron el mismo resultado que 2 capas de 100 neuronas.

**¿Qué inconvenientes hay al encontrar el tamaño óptimo de la red? ¿Por qué?**

Como visto anteriormente: Los inconvenientes de encontrar el tamaño óptimo de la red incluyen el costo computacional y el tiempo necesario para buscar exhaustivamente la mejor arquitectura. Además, existe el riesgo de sobreajuste, especialmente cuando se aumenta la complejidad de la red. Cuantas más capas y neuronas se agregan, mayor es el riesgo de que el modelo se ajuste demasiado a los datos de entrenamiento y no generalice bien con datos nuevos.

En resumen, aunque esperaba una mejora más grande, la modesta mejora en el rendimiento puede deberse a la naturaleza del problema y la cantidad de datos disponibles. Encontrar el tamaño óptimo de la red es un proceso desafiante que debe equilibrar la complejidad de la red con la complejidad del problema y la cantidad de datos disponibles. Es importante tener en cuenta que, a veces, una arquitectura más simple puede ser suficiente para resolver ciertos problemas.